In [1]:
from src.data_processing import *
from src.wav2vec2_french_phonemizer import *
from src.readingTestFluencE_evaluation import *

In [2]:
# We prepare the test data for the readingTestFluencE dataset
readingTestFluencE_df = prepare_readingTestFluencE_data()

/Users/cleo/EPFL/Ma2/Project/Rhythmico-ASR/src/data_processing.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readingTestFluencE_df['testResults'] = readingTestFluencE_df['testResults'].apply(lambda x: convert_str_to_dct_eval(x))
/Users/cleo/EPFL/Ma2/Project/Rhythmico-ASR/src/data_processing.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readingTestFluencE_df['evaluationResults'] = readingTestFluencE_df['evaluationResults'].apply(lambda x: convert_str_to_dct_eval(x))


In [3]:
# The tests id of the 10 selected readingTestFluencE tests
tests_id = [
    '2BB671AA-2F6A-4346-8B76-F0C89C236390',
    '3B545E56-D802-4380-9993-21C11066B12E',
    '5C1C826F-E778-48C3-9170-6BF943175984',
    '046E4FEB-E284-48D5-922E-616DA7651F02',
    '75A80925-F8CF-463D-AFED-5CC399848CC2',
    '102DCD09-43EA-434D-A590-0FA5C7C7C1B3',
    '098522E8-2203-425E-85E5-5809D5B0B523',
    '79055215-1979-42D3-9B26-B9C6DD935D83',
    'ABD81BE7-7629-4816-8241-7ECBF32DFFFA',
]

# We generate the top-3 phoneme transcriptions for the selected tests
top_3_phoneme_transcriptions(tests_id=tests_id)

Filtered top-3 phoneme transcriptions saved to sample_readingTestFluencE/readingTestFluencE_2BB671AA-2F6A-4346-8B76-F0C89C236390_phonemes.csv
Filtered top-3 phoneme transcriptions saved to sample_readingTestFluencE/readingTestFluencE_3B545E56-D802-4380-9993-21C11066B12E_phonemes.csv
Filtered top-3 phoneme transcriptions saved to sample_readingTestFluencE/readingTestFluencE_5C1C826F-E778-48C3-9170-6BF943175984_phonemes.csv
Filtered top-3 phoneme transcriptions saved to sample_readingTestFluencE/readingTestFluencE_046E4FEB-E284-48D5-922E-616DA7651F02_phonemes.csv
Filtered top-3 phoneme transcriptions saved to sample_readingTestFluencE/readingTestFluencE_75A80925-F8CF-463D-AFED-5CC399848CC2_phonemes.csv
Filtered top-3 phoneme transcriptions saved to sample_readingTestFluencE/readingTestFluencE_102DCD09-43EA-434D-A590-0FA5C7C7C1B3_phonemes.csv
Filtered top-3 phoneme transcriptions saved to sample_readingTestFluencE/readingTestFluencE_098522E8-2203-425E-85E5-5809D5B0B523_phonemes.csv
Filter

In [4]:
# The full text of the readingTestFluencE
ground_truth = 'sɛ listwaʁ də məsjø pəti ki vi dɑ̃z yn vjɛj mɛzɔ̃ sitye o kœʁ dœ̃ vjø vilaʒ la mɛzɔ̃ ɛt ɑ̃tuʁe dœ̃ ʒaʁdɛ̃ avɛk yn baʁjɛʁ il i a de kɔ̃kɔ̃bʁ de ʃu fʁize tut sɔʁt də leɡymz o fɔ̃ dy ʒaʁdɛ̃ lə pɔʁtijɔ̃ ʁɛst tuʒuʁ fɛʁme puʁ kə ʃjɛ̃ a pys nə seʃap pa ʃjɛ̃ a pysz ɛm sə kuʃe pʁɛ də la pubɛl a lɔ̃bʁ dœ̃n ɔʁɑ̃ʒe kuvɛʁ də fʁyi delisjø ʃjɛ̃ a pysz ɛ ɡuʁmɑ̃ il kʁɔk tu sə ki lyi pas su la dɑ̃ dez ɔʁɑ̃ʒ puʁi ki tɔ̃b syʁ lə sɔl de flœʁ fanez œ̃ mɔʁso də byvaʁ œ̃ ʒuʁ məsjø pəti desid də mɛtʁ ʃjɛ̃ a pys dɑ̃z yn niʃ ʃjɛ̃ a pys nɛm paz ɛtʁ ɑ̃fɛʁme il pʁefɛʁ sɑ̃dɔʁmiʁ ɑ̃ ʁəɡaʁdɑ̃ lez etwal dɑ̃ lə sjɛl tut le nyiz il abwa kɑ̃ məsjø pəti va sə kuʃe məsjø pəti desid də dɔʁmiʁ dɑ̃ lə ɡʁənje də sa ʒɔli mɛzɔ̃ puʁ pʁɑ̃dʁ œ̃ pø də ʁəpoz il nə tʁuv ply lə sɔmɛj yn nyi dɛ̃sɔmni ɔp il sot dy li e uvʁ la ɡʁɑ̃d mal ki sə tʁuv dəvɑ̃ lyi dɑ̃z œ̃ kwɛ̃ sɔ̃bʁ dy ɡʁənje e la syʁpʁiz tut sa vi kil pɑ̃sɛ sɑ̃z istwaʁ lyi ʁəvjɛ̃t ɑ̃ memwaʁ il sɔʁ le muʃwaʁ bʁɔde paʁ sa ɡʁɑ̃mɛʁ se pətit dɑ̃ də lɛ sɔ̃ po də ʃɑ̃bʁ ebʁeʃe yn tɛt də pwasɔ̃ seʃe œ̃ sak plɛ̃ də bijz yn mɔ̃tʁ ki fɛ tik tak tik tak sɔ̃ kaʁnɛ də nɔtz œ̃ bu də lasɛ sɔ̃ vjø tʁɑ̃zistɔʁ a pil sɛ fu kɔm tu se suvniʁ sə buskyl dɑ̃ sa tɛt e il nə pø ʁətniʁ se laʁm demɔsjɔ̃ sa vi nɛ pa sɑ̃z istwaʁ il sə suvjɛ̃t ɛɡzaktəmɑ̃ də la vwa dy pʁezɑ̃tatœʁ meteo lə tɑ̃ va sameljɔʁe dəmɛ̃ ɑ̃ deby də matine syʁ nɔtʁ ʁeʒjɔ̃ sjɛl ʃaʁʒe lapʁɛmidi il sə ʁapɛl le vjɛj pyblisitez aɛma e la salte sɑ̃ va ɔ̃n a tuʒuʁ bəzwɛ̃ də pəti pwa ʃe swa le pʁəmjɛʁ lymjɛʁ dy ʒuʁ penɛtʁ paʁ la pətit fənɛtʁ dy ɡʁənje il ɛt o kœʁ də se suvniʁ kɑ̃ sɔ̃ ʁevɛj sɔndʁɪŋ dʁɪŋ dʁɪŋ'.split(" ")

buffer_sizes = np.arange(10, 31, 1)

# For each of the tests, we will evaluate the top 3 phoneme transcriptions
for test_id in tests_id:
    print("==" * 20)
    print(f"Test ID: {test_id}")

    # We iterate over different sizes of buffer to evaluate
    for buffer_size in buffer_sizes:
        # We evaluate the test results for each of the tests
        top_3_phoneme_evaluation(readingTestFluencE_df, test_id, ground_truth, buffer_size=buffer_size)

Test ID: 2BB671AA-2F6A-4346-8B76-F0C89C236390
Buffer size: 10

Final Score: 42/57 words read correctly.
Buffer size: 11

Final Score: 45/57 words read correctly.
Buffer size: 12

Final Score: 45/57 words read correctly.
Buffer size: 13

Final Score: 45/57 words read correctly.
Buffer size: 14

Final Score: 44/57 words read correctly.
Buffer size: 15

Final Score: 44/57 words read correctly.
Buffer size: 16

Final Score: 44/57 words read correctly.
Buffer size: 17

Final Score: 44/57 words read correctly.
Buffer size: 18

Final Score: 45/57 words read correctly.
Buffer size: 19

Final Score: 45/57 words read correctly.
Buffer size: 20

Final Score: 45/57 words read correctly.
Buffer size: 21

Final Score: 45/57 words read correctly.
Buffer size: 22

Final Score: 45/57 words read correctly.
Buffer size: 23

Final Score: 46/57 words read correctly.
Buffer size: 24

Final Score: 46/57 words read correctly.
Buffer size: 25

Final Score: 46/57 words read correctly.
Buffer size: 26

Final Sco

In [5]:
# The full text of the readingTestFluencE
ground_truth = 'sɛ listwaʁ də məsjø pəti ki vi dɑ̃z yn vjɛj mɛzɔ̃ sitye o kœʁ dœ̃ vjø vilaʒ la mɛzɔ̃ ɛt ɑ̃tuʁe dœ̃ ʒaʁdɛ̃ avɛk yn baʁjɛʁ il i a de kɔ̃kɔ̃bʁ de ʃu fʁize tut sɔʁt də leɡymz o fɔ̃ dy ʒaʁdɛ̃ lə pɔʁtijɔ̃ ʁɛst tuʒuʁ fɛʁme puʁ kə ʃjɛ̃ a pys nə seʃap pa ʃjɛ̃ a pysz ɛm sə kuʃe pʁɛ də la pubɛl a lɔ̃bʁ dœ̃n ɔʁɑ̃ʒe kuvɛʁ də fʁyi delisjø ʃjɛ̃ a pysz ɛ ɡuʁmɑ̃ il kʁɔk tu sə ki lyi pas su la dɑ̃ dez ɔʁɑ̃ʒ puʁi ki tɔ̃b syʁ lə sɔl de flœʁ fanez œ̃ mɔʁso də byvaʁ œ̃ ʒuʁ məsjø pəti desid də mɛtʁ ʃjɛ̃ a pys dɑ̃z yn niʃ ʃjɛ̃ a pys nɛm paz ɛtʁ ɑ̃fɛʁme il pʁefɛʁ sɑ̃dɔʁmiʁ ɑ̃ ʁəɡaʁdɑ̃ lez etwal dɑ̃ lə sjɛl tut le nyiz il abwa kɑ̃ məsjø pəti va sə kuʃe məsjø pəti desid də dɔʁmiʁ dɑ̃ lə ɡʁənje də sa ʒɔli mɛzɔ̃ puʁ pʁɑ̃dʁ œ̃ pø də ʁəpoz il nə tʁuv ply lə sɔmɛj yn nyi dɛ̃sɔmni ɔp il sot dy li e uvʁ la ɡʁɑ̃d mal ki sə tʁuv dəvɑ̃ lyi dɑ̃z œ̃ kwɛ̃ sɔ̃bʁ dy ɡʁənje e la syʁpʁiz tut sa vi kil pɑ̃sɛ sɑ̃z istwaʁ lyi ʁəvjɛ̃t ɑ̃ memwaʁ il sɔʁ le muʃwaʁ bʁɔde paʁ sa ɡʁɑ̃mɛʁ se pətit dɑ̃ də lɛ sɔ̃ po də ʃɑ̃bʁ ebʁeʃe yn tɛt də pwasɔ̃ seʃe œ̃ sak plɛ̃ də bijz yn mɔ̃tʁ ki fɛ tik tak tik tak sɔ̃ kaʁnɛ də nɔtz œ̃ bu də lasɛ sɔ̃ vjø tʁɑ̃zistɔʁ a pil sɛ fu kɔm tu se suvniʁ sə buskyl dɑ̃ sa tɛt e il nə pø ʁətniʁ se laʁm demɔsjɔ̃ sa vi nɛ pa sɑ̃z istwaʁ il sə suvjɛ̃t ɛɡzaktəmɑ̃ də la vwa dy pʁezɑ̃tatœʁ meteo lə tɑ̃ va sameljɔʁe dəmɛ̃ ɑ̃ deby də matine syʁ nɔtʁ ʁeʒjɔ̃ sjɛl ʃaʁʒe lapʁɛmidi il sə ʁapɛl le vjɛj pyblisitez aɛma e la salte sɑ̃ va ɔ̃n a tuʒuʁ bəzwɛ̃ də pəti pwa ʃe swa le pʁəmjɛʁ lymjɛʁ dy ʒuʁ penɛtʁ paʁ la pətit fənɛtʁ dy ɡʁənje il ɛt o kœʁ də se suvniʁ kɑ̃ sɔ̃ ʁevɛj sɔndʁɪŋ dʁɪŋ dʁɪŋ'.split(" ")

# For each of the tests, we will evaluate the top 3 phoneme transcriptions
for test_id in tests_id:
    print("==" * 20)
    print(f"Test ID: {test_id}")

    # We evaluate the test results for each of the tests
    top_3_phoneme_evaluation(readingTestFluencE_df, test_id, ground_truth, buffer_size=20, detailed=True)

Test ID: 2BB671AA-2F6A-4346-8B76-F0C89C236390
Buffer size: 20
Detailed Evaluation:
✅ C'est → correct
✅ l'histoire → correct
✅ de → correct
❌ Monsieur → missed
✅ Petit → correct
✅ qui → correct
✅ vit → correct
✅ dans → correct
✅ une → correct
❌ vieille → missed
✅ maison → correct
✅ située → correct
✅ au → correct
❌ coeur → missed
❌ d'un → missed
✅ vieux → correct
✅ village. → correct
❌ La → missed
❌ maison → missed
✅ est → correct
✅ entourée → correct
✅ d'un → correct
✅ jardin → correct
✅ avec → correct
✅ une → correct
✅ barrière → correct
✅ ; → correct
✅ il → correct
✅ y → correct
✅ a → correct
❌ des → missed
✅ concombres, → correct
✅ des → correct
✅ choux → correct
✅ frisés, → correct
✅ toutes → correct
✅ sortes → correct
❌ de → missed
✅ légumes. → correct
✅ Au → correct
✅ fond → correct
✅ du → correct
✅ jardin, → correct
❌ le → missed
✅ portillon → correct
✅ reste → correct
✅ toujours → correct
✅ fermé → correct
✅ pour → correct
❌ que → missed
✅ Chien → correct
✅ à → correct
✅ Puces 